<a href="https://colab.research.google.com/github/Aisha-Hagar/LLMs/blob/main/article_title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install -U datasets huggingface_hub fsspec
!pip install rouge_score
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudn

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import evaluate
import nltk
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import pandas as pd
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler

2025-06-24 19:04:28.132624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750791868.300136      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750791868.350830      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
ds = load_dataset("azrai99/the-star-news-articles")
ds

README.md:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

news_v1.csv:   0%|          | 0.00/67.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21709 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 21709
    })
})

In [ ]:
ds['train'][0]

{'content_id': 1375152,
 'title': 'Chinese Premier says Malaysia-China relations exemplary, proposes four steps to advance ties',
 'text': 'KUALA LUMPUR: China will continue to work with Malaysia in advancing their friendship and bilateral relations for mutual benefit and a brighter future for both nations, said its Premier Li Qiang.\n\nSpeaking at a dinner to commemorate the 50th anniversary of Malaysia-China bilateral relations, which was attended by Prime Minister Datuk Seri Anwar Ibrahim here on Wednesday (June 19), Li said the relationship is a fine example of relations between countries in the region.\n\nLi emphasised the milestone as a new starting point for both nations to chart a stronger and more prosperous shared future together.\n\n"Be it in the past, present or future, friendship remains the defining feature of China-Malaysia relations. We have brought this relationship to a new starting point and we share the aspiration to carry it forward to future generations,” he said.

In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> targets: {example['title']}'")
        print(f"'>> task_type: {example['text']}'")


show_samples(ds)


'>> targets: GameStop, AMC dive further as meme stocks rally tapers off'
'>> task_type: (Reuters) -Retail investor darlings GameStop and AMC fell for a second straight session on Thursday, as the excitement over the social media return of "Roaring Kitty", who was the central figure in 2021 meme stock rally, fizzles out.

Shares of the videogame retailer GameStop fell 17% to $32.60 after jumping as much as $64.83 this week. Theater chain AMC shed 11% following an 88% surge since Friday's close.

Both the shares were to set to shed half of their gains made in the first two sessions of the week following a series of posts from Keith Gill's X account "Roaring Kitty", whose bullish calls on GameStop was a reason for the 2021 meme stocks frenzy.

Unlike 2021 when Reddit users banded together to target highly shorted stocks that burnt the bearish bets of hedge funds, institutional investors too are part of the meme stock mania, Vanda Research, which tracks retail investor flows, said.

"The 

In [ ]:
ds = ds.remove_columns(['content_id', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'])
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 21709
    })
})

In [ ]:
ds = ds["train"].train_test_split(test_size=0.2)
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 17367
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 4342
    })
})

In [ ]:
ds_tst = ds["test"].train_test_split(test_size=0.5)
ds_tst

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 2171
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 2171
    })
})

In [ ]:
ds["validation"] = ds_tst["train"]
ds["test"] = ds_tst["test"]
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 17367
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 2171
    })
    validation: Dataset({
        features: ['title', 'text'],
        num_rows: 2171
    })
})

In [ ]:
l = []
for t in ds['train']['text']:
  l.append(len(t.split()))

In [ ]:
min(l)

26

In [ ]:
ds = ds.filter(lambda x: len(x["title"].split()) > 2)
ds

Filter:   0%|          | 0/17367 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2171 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2171 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 17336
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 2168
    })
    validation: Dataset({
        features: ['title', 'text'],
        num_rows: 2169
    })
})

In [ ]:
ds.save_to_disk("Data")

Saving the dataset (0/1 shards):   0%|          | 0/17336 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2168 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2169 [00:00<?, ? examples/s]

In [ ]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
max_input_length = 600
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/17336 [00:00<?, ? examples/s]

Map:   0%|          | 0/2168 [00:00<?, ? examples/s]

Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

In [ ]:
rouge_score = evaluate.load("rouge")

In [ ]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [ ]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(ds["train"][1]["text"]))

KUALA LUMPUR: Potential exports worth RM2.4bil from businesses in Kazakhstan and Uzbekistan indicate numerous opportunities for Malaysian companies to explore the region.
“This significant figure reflects the growing trust of their respective business communities in Malaysia as a strategic trade and investment partner,” said Investment, Trade and Industry (Miti) Minister Tengku Datuk Seri Zafrul Aziz.
Tengku Zafrul assured that Miti will expedite the realisation of these export commitments to support Malaysia’s gross domestic growth.


In [ ]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["text"]]
    return metric.compute(predictions=summaries, references=dataset["title"])

In [ ]:
score = evaluate_baseline(ds["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 13.82, 'rouge2': 4.78, 'rougeL': 11.34, 'rougeLsum': 12.29}

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    ds["train"].column_names
)

In [ ]:
tokenized_datasets.set_format("torch")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [ ]:
output_dir = 'Model'

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
    #     repo.push_to_hub(
    #         commit_message=f"Training in progress epoch {epoch}", blocking=False
    #     )

  0%|          | 0/21670 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 0: {'rouge1': 11.492, 'rouge2': 2.5773, 'rougeL': 10.5321, 'rougeLsum': 10.5355}
Epoch 1: {'rouge1': 13.6343, 'rouge2': 3.5921, 'rougeL': 12.3527, 'rougeLsum': 12.3471}
Epoch 2: {'rouge1': 14.4095, 'rouge2': 4.2443, 'rougeL': 13.668, 'rougeLsum': 13.6514}
Epoch 3: {'rouge1': 21.9202, 'rouge2': 7.778, 'rougeL': 20.1693, 'rougeLsum': 20.1673}
Epoch 4: {'rouge1': 24.0384, 'rouge2': 8.9592, 'rougeL': 22.0477, 'rougeLsum': 22.0835}
Epoch 5: {'rouge1': 25.542, 'rouge2': 9.7864, 'rougeL': 23.4558, 'rougeLsum': 23.4887}
Epoch 6: {'rouge1': 26.5858, 'rouge2': 10.4584, 'rougeL': 24.3976, 'rougeLsum': 24.4449}
Epoch 7: {'rouge1': 27.3738, 'rouge2': 10.7457, 'rougeL': 25.0139, 'rougeLsum': 25.0646}


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=10000.0 (msgs/sec)
NotebookApp.rate_limit_window=1.0 (secs)



Epoch 8: {'rouge1': 27.8806, 'rouge2': 11.1092, 'rougeL': 25.422, 'rougeLsum': 25.4615}
Epoch 9: {'rouge1': 27.9126, 'rouge2': 11.1549, 'rougeL': 25.4242, 'rougeLsum': 25.4422}


In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM , AutoTokenizer
from transformers import AutoConfig


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('Model')
tokenizer = AutoTokenizer.from_pretrained('Model')

summarizer = pipeline(task='summarization',  # replace with whatever task you have
                model=model,
                tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
config = AutoConfig.from_pretrained("Model")
print(config.architectures)

['MT5ForConditionalGeneration']


In [ ]:
def print_title(idx):
    article = ds["test"][idx]["text"]
    title = ds["test"][idx]["title"]
    gen_title = summarizer(ds["test"][idx]["text"])[0]["summary_text"]
    print(f"'>>> Article: {article}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Generated Title: {gen_title}'")

In [ ]:
print_title(10)

'>>> Article: PETALING JAYA: Flights from Kuala Lumpur to destinations in Sabah and Sarawak have been cancelled due to the eruption of Mount Ruang in Indonesia, says Malaysia Airlines (MAB).

It said this in a post on its official Facebook page, in which it gave a list of all flights affected. These flights are;

To/from Sabah

MH7420 (KUL-TWU)

MH7421 (TWU-KUL)

MH2612 (KUL-BKI)

MH2621 (BKI-KUL)

MH2710 (KUL-SDK)

MH2711 (SDK-KUL)

MH2610 (KUL-BKI)

MH2611 (BKI-KUL)

MH7404 (KUL-BKI)

MH7405 (BKI-KUL)

MH2613 (BKI-KUL)

To/from Sarawak

MH2520 (KUL-KCH)

MH2513 (KCH-KUL)

MH2542 (KUL-KCH)

MH2543 (KCH-KUL)

MH2574 (KUL-MYY)

MH2575 (MYY-KUL)

MH2742 (KUL-BTU)

MH2473 (BTU-KUL)

"Concurrently, the airline is working to accommodate passengers affected by the cancellation of flights on alternative flights once the situation progressively improves," added MAB.

It said that it is continuing to closely monitor developments and will issue further updates on flight cancellations on its soci

In [ ]:
print_title(0)

'>>> Article: CONSUMERS have been advised to be more selective when eating out in view of escalating prices of food in recent months.

Johor Consumers Movement Association chairman Md Salleh Sadijo said consumerism awareness among Malaysians was still low.

“Lack of interest or concern is the reason why consumers are easily targeted by unscrupulous traders,” he said.

Md Salleh said traders were taking advantage of the situation, knowing that consumers would complain before eventually accepting the price increases.

He urged Malaysian consumers to start taking matters seriously including boycotting businesses and traders that engaged in excessive profiteering.

“If we can boycott companies and brands that are openly supporting Israel, we also can do the same against our local traders for excessive profiteering,” said Md Salleh.

He said this after complaints that certain eateries in Johor Baru had increased their prices following the May 1 announcement by Prime Minister Datuk Seri Anwa

In [ ]:
print_title(100)

'>>> Article: “I don't think things are going to work out between us.” It’s a sentence that seems too final, too absolute. What if my date from last weekend thinks I’m arrogant for writing a text like that?

Instead, maybe not texting or talking to them at all will make things easier. But for whom? Mostly just for the one doing the ghosting.

It can happen to anyone, and Charlie’s Angels star and talk show host Drew Barrymore revealed in a 2022 interview with People magazine that she, too, had been a victim.

Early in the 2000s, the term “ghosting” emerged in the Western dating scene to describe when someone suddenly cuts off all communication and contact without any warning or explanation.

Ghosting is not the same as a breakup, says psychologist Anja Wermann, who herself was ghosted at the end of 2020.

“When you split up, you usually see it coming, you might have a conversation to end the relationship. When it comes to ghosting, however, the reason for ending things is never given,”

In [ ]:
pred = summarizer(ds["test"]["text"])

In [ ]:
preds = [j for i in pred for j in i.values()]
preds

['Traders advised to be more selective when eating out',
 'Taylor’s University Bachelor of Fashion Design Technology graduates demonstrate fashion design for six semesters',
 'Soccer-Penang students began 2024/25 school session',
 'Tennis-Djokovic hopes to shake off freak head injury',
 'Soccer-Westports Holdings shares ahead of growth prospects',
 'Soccer teacher Joohari Ariffin announces unity government candidate for state seat',
 'Trump selected 12 jurors who will assess guilt or innocence over coming weeks',
 'Soccer-actress Jia Ling named Prada ambassador',
 'Soccer-Belgium left back Thomas Meunier 10 days away from full fitness',
 'WCT shares remains partial stake in Segi Astana',
 'Malaysia Airlines cancelles flights from Kuala Lumpur to destinations in Sabah and Sarawak',
 "Soccer-Bourin says 'The more anger' in Caribbean'",
 'Japan’s exports struggled as soft demand weighs on growth',
 'Australian lammos can be made in a variety of flavours',
 'Motor-driven Proton X50 offers 

In [ ]:
len(preds)

2168

In [ ]:
y_test = ds['test']['title']
y_test

['‘Avoid eateries that hike prices excessively’',
 'Let the show begin',
 'New school session begins for over 217,000 in Penang',
 'Tennis-Headaches aplenty for Djokovic before French Open title defence',
 'Westports poised to benefit from trade recovery',
 'Sg Bakap polls: Unity government names Dr Joohari Ariffin as candidate',
 'Lawyers select 12 jurors to serve in Trump hush-money case',
 "Chinese actress Jia Ling, who lost 50kg for 'Yolo', is Prada brand ambassador",
 'Soccer-Belgium keep faith with Meunier for European Championship',
 'Divestment in gateway@KLIA2 to put WCT on stronger footing',
 'Malaysia Airlines cancels Sabah, Sarawak flights due to Mt Ruang eruption',
 "Descendants of enslaved, enslavers 'break silence' around France's past",
 'Exports rise but soft demand weighs on growth',
 'Traditional Aussie cake with French twist',
 'Proton eyes monthly sales of up to 2,500 units of 2024 Proton X50',
 'South Korean AI chip developers Rebellions and Sapeon to merge',
 '‘W

In [ ]:
len(y_test)

2168

In [ ]:
decoded_preds, decoded_labels = postprocess_text(preds, y_test)

In [ ]:
rs = rouge_score.compute(predictions=decoded_preds, references=decoded_labels)

In [ ]:
rs

{'rouge1': 0.2835371452398944,
 'rouge2': 0.1127414042161173,
 'rougeL': 0.2587176175108021,
 'rougeLsum': 0.2586369805110029}